# Programming an LLM-based chatbot
In this notebook we will program a basic chatbot. We can use multiple large language models, via the [OpenAI Python interface](https://github.com/openai/openai-python).

In [1]:
import openai
openai.__version__

'1.5.0'

## OpenAI's chatGPT
We can use OpenAI's chatGPT via its pay-per-use model by signing up [here](https://openai.com/index/openai-api).

In [2]:
def prompt_chatGPT(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    import os
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
        
    # setup connection to the LLM
    client = openai.OpenAI()
    # todo: enter your API key here:
    # client.api_key = ""
    client.api_key = os.environ.get('OPENAI_API_KEY')
    
    # submit prompt
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    
    # extract answer
    return response.choices[0].message.content

In [3]:
prompt_chatGPT("Hi!")

'Hello! How can I help you today?'

## Ollama

Alternatively, we can use [ollama](https://ollama.com/download), a tool that downloads models to our computer and allows us to run them locally. Before executing the following code, you need to run `ollama run gemma:2b` once.

In [4]:
def prompt_ollama(message:str, model="gemma:2b"):
    """A prompt helper function that sends a message to ollama
    and returns only the text response.
    """
    import os
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
        
    # setup connection to the LLM
    client = openai.OpenAI()
    client.base_url = "http://localhost:11434/v1"
    client.api_key = "none"
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    
    # extract answer
    return response.choices[0].message.content

In [5]:
prompt_ollama("Hi!")

'Hello! How can I assist you today?\n\nWhat would you like to talk about?'

## Blablador

Another alternative is to use the [blablardor](https://helmholtz-blablador.fz-juelich.de/) infrastructure at the Research Center Jülich. Before you can access it, you need to create an API key as explained [on this page](https://sdlaml.pages.jsc.fz-juelich.de/ai/guides/blablador_api_access/).

In [6]:
def prompt_blablador(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to Blablador (FZ Jülich)
    and returns only the text response.
    """
    import os
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
    
    # setup connection to the LLM
    client = openai.OpenAI()
    client.base_url = "https://helmholtz-blablador.fz-juelich.de:8000/v1"
    
    # todo: enter your API key here:
    # client.api_key = ""
    client.api_key = os.environ.get('BLABLADOR_API_KEY')
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    
    # extract answer
    return response.choices[0].message.content

In [7]:
prompt_blablador("Hi!")

'Hello! How can I assist you today?'

## Decide for a prompt function
Here we need to decide for one of the prompt-functions above.

In [8]:
prompt = prompt_chatGPT

In [9]:
prompt("Hi!")

'Hello! How can I assist you today?'

## Adding memory
Our prompt function does not have memory yet.

In [10]:
prompt("Hi, my name is Robert.")

'Hello Robert, nice to meet you! How can I assist you today?'

In [11]:
prompt("What is my name?")

"I'm sorry, I don't have access to your personal information and cannot determine your name. You would need to provide me with your name for me to know it."

Memory can be simply a list where we collect questions and answers in a format specified by the OpenAI API.

In [12]:
chat_history = []
def prompt_with_memory(message:str):
    
    # convert message in the right format and store it in memory
    question = {"role": "user", "content": message}
    chat_history.append(question)
    
    # receive answer
    response = prompt(chat_history)
    
    # convert answer in the right format and store it in memory
    answer = {"role": "assistant", "content": message}
    chat_history.append(answer)
    
    return response

In [13]:
prompt_with_memory("Hi, my name is Robert.")

'Nice to meet you, Robert. How can I assist you today?'

In [14]:
prompt_with_memory("What is my name?")

'Your name is Robert.'

## Continuous discussion

Finally, we use the functions above for a continuous chat.

In [15]:
question = ""
while(question != "bye"):
    question = input("Q:")
    
    answer = prompt_with_memory(question)
    print("A:",answer)

Q: hi my name is Alice


A: Nice to meet you, Alice! How can I assist you today?


Q: what is my name?


A: Your name is Alice.


Q: How can I denoise an image using Python?


A: You can denoise an image in Python using libraries such as OpenCV or scikit-image. One common method for denoising images is using the bilateral filter, which preserves edges while smoothing the rest of the image. Here's an example using OpenCV:

```python
import cv2

# Load the image
image = cv2.imread('image.jpg')

# Denoise the image using the bilateral filter
denoised_image = cv2.bilateralFilter(image, 9, 75, 75)

# Display the denoised image
cv2.imshow('Denoised Image', denoised_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

This code will load an image, apply the bilateral filter for denoising, and display the denoised image. You can adjust the parameters of the bilateral filter (like the kernel size and sigma values) to achieve the desired level of denoising.


Q: bye


A: Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!
